# Open AI Function Calling

In [ ]:
%pip install openai

In [1]:
import os.path
import requests

# Download einer Beispiel-Speisekarte, welche bereits in strukturiertem Python-Code vorliegt.
# Könnte man später durch einen Datei- oder Webseitenimport ersetzen.

MENU_DEMO_URL = "https://raw.githubusercontent.com/TAS-AG/TAS.OpenAI.Workshop/refs/heads/main/data/speisekarte_pizza_sample.py"
MENU_MICELLOS_URL = "https://raw.githubusercontent.com/TAS-AG/TAS.OpenAI.Workshop/refs/heads/main/data/speisekarte_micellos_leipzig.pdf"
MENU_ATRIUM_URL = "https://raw.githubusercontent.com/TAS-AG/TAS.OpenAI.Workshop/refs/heads/main/data/wochenkarte_bistro_im_atrium.pdf"

download_urls = [MENU_DEMO_URL, MENU_MICELLOS_URL, MENU_ATRIUM_URL]

for url in download_urls:
    response = requests.get(url)
    filename = os.path.basename(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

from speisekarte_pizza_sample import MENU

In [2]:
import base64

with open("speisekarte_micellos_leipzig.pdf", "rb") as f:
    data = f.read()

speisekarte_micellos_base64_string = base64.b64encode(data).decode("utf-8")


In [ ]:
from openai import OpenAI
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = OpenAI(api_key = openai_api_key)

Multimodale Modelle sind KI-Modelle, die Informationen aus verschiedenen Modalitäten – etwa Text, Bilder, Audio oder strukturierte Daten – gleichzeitig verarbeiten und kombinieren können. Das unterscheidet sie von klassischen Modellen, die nur mit einer einzigen Eingabemodalität (z. B. nur Text) umgehen können.

GPT 4o beispielsweise kann multimodalen Input wie Text, PDFs oder Bilder verarbeiten. Im folgenden Beispiel verwenden wir gpt-4o zur Verarbeitung eines Nutzer-Inputs und einer PDF-Datei (z. B. einer Speisekarte).

Aufgabe: Neben Speisen für die Pizzeria soll der Kellner auch Speisen für das Restaurant "Bistro im Atrium" in Leipzig aufnehmen können. Erweitere den API Aufruf, sodass beide Speisekarten übergeben werden. Füge im Systemprompt den aktuellen Tag hinzu und mache Anpassungen, dass nur die für den Tag verfügbaren Speisen des Atrium-Bistros berücksichtigt werden. Erweitere das Output-Schema so, dass auch das entsprechende Restaurant ("Atrium im Bisto" oder "Micellos") in der Ausgabe enthalten sind. Finde Bestellungen für die Speisen aus beiden Restaurants empfohlen werden. Passe den Prompt wo nötig an.

In [ ]:
import json

user_input = "Was gibt es diese Woche im Atrium-Bistro?."

expected_response_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",  #Erwarteter Rückgabetyp ist ein Array von Objekten
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

response = openai_client.responses.create(
    model = "gpt-4o",
    input = [
        {
            "role": "developer",
            "content": [
                {
					"type": "input_text",
					"text": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."
				}
            ]
        },
        {
            "role": "user",
            "content":  [
                {
                    "type": "input_file",
                    "filename": "speisekarte_micellos_leipzig.pdf",
                    "file_data": f"data:application/pdf;base64,{speisekarte_micellos_base64_string}"
                },
                {
                    "type": "input_text",
                    "text": f"{user_input}",
                }
			]
		}
    ],
    # Gewünschtes Ausgabeformat wird API-Aufruf übergeben
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": expected_response_schema,
            "strict": True
        }
    }
)
recommendations = json.loads(response.output_text)["items"]

for item in recommendations:
    print(item)


BadRequestError: Error code: 400 - {'error': {'message': 'The file type you uploaded is not supported. Please try again with a pdf', 'type': 'invalid_request_error', 'param': 'input', 'code': 'unsupported_file'}}